In [ ]:
from langchain_community.llms import Ollama
from langchain_ollama import ChatOllama
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
llm = ChatOllama(model="llama3.2:latest", temperature=0)

In [ ]:
def reverse_string(s):
    """Reverses the input string."""
    return s[::-1]

def capitalize_string(s):
    """Capsitizes the input string."""
    return s[:1].upper() + s[1:]

def to_lowercase(s):
    """Converts the input string to lowercase."""
    return s.lower()

def to_uppercase(s):
    """Converts the input string to uppercase."""
    return s.upper()


In [ ]:
tools = [reverse_string, capitalize_string, to_lowercase, to_uppercase]
llm_with_tools = llm.bind_tools(tools)

In [ ]:
sys_msg = SystemMessage(content="Your are a helpful assistant tasked with performing string functions")


In [ ]:
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

#### Build Agent

In [ ]:
# Graph
builder = StateGraph(MessagesState)
memory = MemorySaver()

# Define nodees: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edgesL these determine how the flow moves

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition,)

builder.add_edge("tools", "assistant")

react_graph = builder.compile(checkpointer=memory)


In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [13]:
messages = [HumanMessage(content="Reverse this string 'Charles McCullough'")]
messages = react_graph.invoke({"messages": messages}, config)

for m in messages['messages']:
    m.pretty_print()


================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (3e293ef4-8a3e-4812-a5f6-373c8bdabbe3)
 Call ID: 3e293ef4-8a3e-4812-a5f6-373c8bdabbe3
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

The reversed string is: hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9)
 Call ID: 29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9
  Args:
    s: hguolluCcM selrahC
================================= Tool Mess

In [14]:
messages = [HumanMessage(content="Now convert the reversed string to upper case.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (3e293ef4-8a3e-4812-a5f6-373c8bdabbe3)
 Call ID: 3e293ef4-8a3e-4812-a5f6-373c8bdabbe3
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

The reversed string is: hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9)
 Call ID: 29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9
  Args:
    s: hguolluCcM selrahC
================================= Tool Mess

In [15]:
messages = [HumanMessage(content="Now reverse the string again and convert it to lower case.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (3e293ef4-8a3e-4812-a5f6-373c8bdabbe3)
 Call ID: 3e293ef4-8a3e-4812-a5f6-373c8bdabbe3
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

The reversed string is: hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9)
 Call ID: 29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9
  Args:
    s: hguolluCcM selrahC
================================= Tool Mess

In [16]:
messages = [HumanMessage(content="Now capitize the string.")]
messages = react_graph.invoke({"messages": messages}, config)
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Reverse this string 'Charles McCullough'
================================== Ai Message ==================================
Tool Calls:
  reverse_string (3e293ef4-8a3e-4812-a5f6-373c8bdabbe3)
 Call ID: 3e293ef4-8a3e-4812-a5f6-373c8bdabbe3
  Args:
    s: Charles McCullough
================================= Tool Message =================================
Name: reverse_string

hguolluCcM selrahC
================================== Ai Message ==================================

The reversed string is: hguolluCcM selrahC
================================ Human Message =================================

Now convert the reversed string to upper case.
================================== Ai Message ==================================
Tool Calls:
  to_uppercase (29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9)
 Call ID: 29ef96f1-536e-4ea2-9bd0-9c33c8e8e3e9
  Args:
    s: hguolluCcM selrahC
================================= Tool Mess